In [1]:
import os
import tensorflow_hub as hub
import timm
from transformers import AutoImageProcessor
from timm import create_model
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from tensorflow import keras
#os.environ["TFHUB_CACHE_DIR"] = "/kaggle/working"
print(os.listdir("../input/effnet2/tensorflow2/default/1"))

2025-05-25 16:37:39.815529: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748191060.017107      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748191060.072085      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


['eff_b4']


In [2]:
def build_mobilenet3(img_size=(224, 224), weights = ""):
    mobile_net = hub.KerasLayer(weights, trainable=False)
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(shape=img_size + (3,)),
        tf.keras.layers.Lambda(lambda x: mobile_net(x))
    ])
    return model
path = "/kaggle/input/cassava-leaf-disease-classification/"
image_path = path+"test_images/"
used_models_keras = { "mobilenet": "../input/cropnet/tensorflow2/classifier-cassava-disease-v1/1",
                     "efficientnetb4" : "../input/effnet2/tensorflow2/default/1/eff_b4"}
submission_df = pd.DataFrame(columns=["image_id","label"])
submission_df["image_id"] = os.listdir(image_path)
submission_df["label"] = 0

In [3]:
from keras.layers import TFSMLayer
import cv2
onlykeras = False
IMAGE_SIZE = (512,512)

def image_augmentations(image):
    p_spatial = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    if p_spatial > 0.75:
        image = tf.image.transpose(image)
        
    if p_rotate > 0.75:
        image = tf.image.rot90(image, k = 3)
    elif p_rotate > 0.5:
        image = tf.image.rot90(image, k = 2)
    elif p_rotate > 0.25:
        image = tf.image.rot90(image, k = 1)

    image = tf.image.resize(image, size = IMAGE_SIZE)
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    
    return image

def create_image_tiles(origin_dim, processed_img):
    crop_size = 512
    img_list = []
    # Cut image into 4 overlapping patches
    for x in [0, origin_dim[1] - crop_size]:
        for y in [0, origin_dim[0] - crop_size]:
            img_list.append(processed_img[x:x+crop_size , y:y+crop_size,:])
    # Keep one additional center cropped image 
    img_list.append(cv2.resize(processed_img[:, 100:700 ,:], dsize=(crop_size, crop_size)))
    return np.array(img_list)

def distribute_unknown(propabilities):
    return propabilities[:,:-1] + np.expand_dims(propabilities[:,-1]/5, 1)

def cut_crop_image(processed_img):
    image = tf.image.central_crop(processed_img, 0.8)
    image = tf.image.resize(image, (224, 224))
    return np.expand_dims(image, 0)

def read_preprocess_file(img_path, normalize=False):
    image = Image.open(img_path)
    if normalize:
        img_scaled = np.array(image)/ 255.0
    else:
        img_scaled = np.array(image)
    img_scaled = img_scaled.astype(np.float32)
    return (image.size[0], image.size[1]), img_scaled

def augment_tiles_light(tiles, ttas=2):
  # Copy central croped image to have same ratio to augmented images
  holdout = np.broadcast_to(tiles[-1,:,:,:],(ttas,) + tiles.shape[1:])
  augmented_batch = tf.map_fn(lambda x: image_augmentations(x), tf.concat(
      [tiles[:-1,:,:,:] for _ in range(ttas)], axis=0))
  return tf.concat([augmented_batch, holdout], axis=0)

def multi_predict_keras(img_path, modelinstance, *args):
    augmented_batch = augment_tiles_light(create_image_tiles(
        *read_preprocess_file(img_path)))
    Yhat = modelinstance(augmented_batch)
    Yhat = Yhat['dense'].numpy()
    print(np.mean(Yhat, axis=0))
    return np.mean(Yhat, axis=0)

def multi_predict_tfhublayer(img_path, modelinstance):
    img = cut_crop_image(read_preprocess_file(img_path, True)[1])
    yhat = tf.nn.softmax(modelinstance.predict(img))
    print(np.mean(distribute_unknown(yhat), axis=0))
    return np.mean(distribute_unknown(yhat), axis=0)

def predict_and_vote(image_list, modelinstances, onlykeras):
    predictions = [] 
    with tqdm(total=len(image_list)) as process_bar:       
      for img_path in image_list:
        process_bar.update(1)  
        Yhats = np.vstack([func(img_path, modelinstance) for func, modelinstance in modelinstances])
        if onlykeras:
            predictions.append(np.argmax(np.sum(Yhats, axis=0)))
        else:
            predictions.append(Yhats)    
    return predictions

inference_models = []

if "mobilenet" in used_models_keras:
    model_mobilenet = build_mobilenet3(weights=used_models_keras["mobilenet"])
    inference_models.append((multi_predict_tfhublayer, model_mobilenet))

if "efficientnetb4" in used_models_keras:
    # model_efficientnetb4 =  keras.models.load_model(used_models_keras["efficientnetb4"], compile=False)
    model_efficientnetb4 = TFSMLayer(used_models_keras["efficientnetb4"], call_endpoint="serving_default")
    inference_models.append((multi_predict_keras, model_efficientnetb4))

submission_df["label"] = predict_and_vote([image_path+id for id in submission_df["image_id"].values], inference_models, onlykeras)

I0000 00:00:1748191087.277783      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
  0%|          | 0/1 [00:00<?, ?it/s]WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1748191146.702217      95 service.cc:148] XLA service 0x79eca0010a90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748191146.702792      95 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1748191147.287768      95 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


I0000 00:00:1748191150.907708      95 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


[0.159776   0.1597643  0.3459106  0.16000386 0.17454527]


100%|██████████| 1/1 [00:08<00:00,  8.77s/it]

[0.11379483 0.12324865 0.22517207 0.09481756 0.44296688]


In [4]:
for i in range(len(submission_df["label"])):
    submission_df.loc[i,"label"] = np.argmax(submission_df["label"][i][0], axis=0)

In [5]:
submission_df[["image_id","label"]].to_csv("submission.csv", index=False)
!head submission.csv

image_id,label
2216849948.jpg,2
